In [1]:
from osgeo import gdal
import numpy as np
import s3fs
import os
import os.path
import netCDF4

In [2]:
# Adjust the variable inputs to create the prefix for the file of interest
bucket_name = 'noaa-goes16' # Change to 'noaa-goes17' for GOES-17 satellite
product_name = 'ABI-L1b-RadM' # This script is ONLY designed for mesoscale products
year = 2020 
day_of_year = 296 
hour = 16
mesoscale = "M1" # Change to "M2" for Mesoscale Domain Sector 2

# Adjust pathname to local directory
directory = "/Users/daniellelosos/"

In [3]:
# Make new directories to store files
if not os.path.isdir(product_name +"_netCDFs"):
    os.mkdir(product_name +"_netCDFs")
if not os.path.isdir(product_name +"_GeoTiffs"):
    os.mkdir(product_name +"_GeoTiffs") 

In [4]:
# Enable GDAL/OGR exceptions
gdal.UseExceptions()

# Use anonymous credentials to access public data  from AWS
fs = s3fs.S3FileSystem(anon=True)

# Write prefix for the files of interest, and list all files beginning with this prefix.
prefix = f'{bucket_name}/{product_name}/{year}/{day_of_year:03.0f}/{hour:02.0f}/'
print(prefix)
files = fs.ls(prefix)

noaa-goes16/ABI-L1b-RadM/2020/296/16/


In [5]:
# Function converts netCDF files to GeoTiffs

def convert(): 
    file_list = []
    # Add only Mesoscale 1 (or 2) ABI Channel i files to the file list
    for file in files:
        name = str(file)
        if mesoscale in name:
            if channel in name:
                file_list.append(file)
    # Select the first file in the list (closest to the top of the hour) and download
    # When scanning in Mode 6, ABI produces 60 Mesoscale 1 and 2 images every hour
    first_file = file_list[0]
    path = directory + product_name + "_netCDFs/" + first_file.split("/")[-1]
    fs.download(first_file, path)
    # Read metadata of ABI Channel 01 netCDF file
    if i == 1:
        dataset = gdal.Open(path, gdal.GA_ReadOnly)
        print ("DATASET:\n" + first_file)
        print ("\nSUBDATASETS:")
        subdatasets = dataset.GetSubDatasets()
        for fname, name in subdatasets:
            print (name)
            print (fname)
        f = netCDF4.Dataset(path)
        global var
        # Choose the first variable/subdataset in the netCDF file to convert to a GeoTiff. 
        var = list(f.variables.keys())[0] 
        print ("\nSelected variable: " + var)
        print ("\nConversion progress:")
    layer = gdal.Open("NETCDF:{0}:{1}".format(path, var))
    tiff_name = product_name + "_" + channel + "_" + var + ".tif"
    geotiff_pathname = os.path.join(directory + product_name +"_GeoTiffs", tiff_name)
    # Make the conversion from a netCDF to a GeoTiff, and store new files in a new folder
    geotiff = gdal.Translate(geotiff_pathname, layer, format = "NetCDF", unscale = True)
    print(tiff_name + " has been generated")
    

In [6]:
# Loop through the sixteen ABI channels.
# For each channel, run the convert function (which creates a GeoTiff of the first image produced at the given hour)
for i in range(1,17):
    if i < 10:
        number = str(i)
        channel = "C0"+ number
        convert()
    if i >= 10:
        number = str(i)
        channel = "C"+ number
        convert()
else:
    print("\n" + product_name + " netCDF files have been converted to GeoTiffs for all 16 ABI channels")

DATASET:
noaa-goes16/ABI-L1b-RadM/2020/296/16/OR_ABI-L1b-RadM1-M6C01_G16_s20202961600274_e20202961600331_c20202961600383.nc

SUBDATASETS:
[1000x1000] toa_outgoing_radiance_per_unit_wavelength (16-bit integer)
NETCDF:"/Users/daniellelosos/ABI-L1b-RadM_netCDFs/OR_ABI-L1b-RadM1-M6C01_G16_s20202961600274_e20202961600331_c20202961600383.nc":Rad
[1000x1000] status_flag (8-bit integer)
NETCDF:"/Users/daniellelosos/ABI-L1b-RadM_netCDFs/OR_ABI-L1b-RadM1-M6C01_G16_s20202961600274_e20202961600331_c20202961600383.nc":DQF

Selected variable: Rad

Conversion progress:
ABI-L1b-RadM_C01_Rad.tif has been generated
ABI-L1b-RadM_C02_Rad.tif has been generated
ABI-L1b-RadM_C03_Rad.tif has been generated
ABI-L1b-RadM_C04_Rad.tif has been generated
ABI-L1b-RadM_C05_Rad.tif has been generated
ABI-L1b-RadM_C06_Rad.tif has been generated
ABI-L1b-RadM_C07_Rad.tif has been generated
ABI-L1b-RadM_C08_Rad.tif has been generated
ABI-L1b-RadM_C09_Rad.tif has been generated
ABI-L1b-RadM_C10_Rad.tif has been generated